In [ ]:
import sys; sys.path.append('..')
import numpy as np, elastic_rods
from bending_validation import suppress_stdout
from linkage_vis import LinkageViewer
from matplotlib import pyplot as plt

l = elastic_rods.RodLinkage('../../examples/etienne_hyperboloid_measured.obj')
driver=l.centralJoint()

l.setMaterial(elastic_rods.RodMaterial('+', 2000, 0.3, [10, 10, 1, 1]))
with suppress_stdout(): elastic_rods.restlen_solve(l)
jdo = l.dofOffsetForJoint(driver)
fixedVars = list(range(jdo, jdo + 6)) # fix rigid motion for a single joint
with suppress_stdout(): elastic_rods.compute_equilibrium(l, fixedVars=fixedVars)

view = LinkageViewer(l, width=512, height=384)
view.show()

In [ ]:
from open_linkage import open_linkage
def equilibriumSolver(tgtAngle, l, opts, fv):
    return elastic_rods.compute_equilibrium(l, tgtAngle, options=opts, fixedVars=fv)
elastic_rods.benchmark_reset()
with suppress_stdout(): cr = open_linkage(l, driver, 6 * np.pi / 16, 50, view, zPerturbationEpsilon=0, equilibriumSolver=equilibriumSolver, maxNewtonIterationsIntermediate=30, verbose=True, useTargetAngleConstraint=True)
elastic_rods.benchmark_report()

In [ ]:
angles = cr[2][:-1]
energies = [r.energy[-1] for r in cr[0]][:-1]
plt.plot(angles, energies)
plt.show()

In [ ]:
plt.plot(angles, np.gradient(np.array(energies), edge_order=2))
plt.show()

In [ ]:
plt.plot(angles, np.gradient(np.gradient(np.array(energies), edge_order=2), edge_order=2))
plt.ylim([-5, 10])
plt.grid()
plt.show()

In [ ]:
import deployment_path_analysis
dpa = deployment_path_analysis.deploymentPathAnalysis(l)
deployment_path_analysis.validateEnergyIncrements(l, epsMax=1e-2)

In [ ]:
deployAngle=6 * np.pi / 16

In [ ]:
openingAngles, stiffnessGaps = deployment_path_analysis.stiffnessGapThroughoutDeployment(l, deployAngle, 5)
view.update()

In [ ]:
import compute_vibrational_modes
fixedVarsWithoutActuator = fixedVars[:]
lambdas, modes = compute_vibrational_modes.compute_vibrational_modes(l, fixedVars=[], mtype=compute_vibrational_modes.MassMatrixType.FULL, n=16, sigma=-1e-6)

import mode_viewer, importlib
importlib.reload(mode_viewer);
mview = mode_viewer.ModeViewer(l, modes, lambdas, amplitude=1.0)
mview.show()

In [ ]:
view.update()

In [ ]:
from matplotlib import pyplot as plt
plt.plot(openingAngles, stiffnessGaps)
plt.ylabel('Relative Deployment Stiffness Gap')
plt.xlabel('Opening Angle')
plt.show()

In [ ]:
np.min(stiffnessGaps)

In [ ]:
stiffnessGaps

In [ ]:
m = deployment_path_analysis.deploymentModeViewer(l)
m.setAmplitude(0.1)
m.show()